<a href="https://colab.research.google.com/github/jayaliyev/nq_hourly-sweep-statistics/blob/main/nq_hourly-sweep-statistics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import zipfile
import glob
import pandas as pd
# import ace_tools

# 1. Extract and load


csv_path = '/content/nq-1m.csv'
df = pd.read_csv(
    csv_path,
    sep=';',
    names=['Date','Time','Open','High','Low','Close','Volume'],
    header=0
)

# 2. Parse datetime and adjust timezone
df['Datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'], format='%d/%m/%Y %H:%M:%S')
df['Datetime'] = df['Datetime'].dt.tz_localize('UTC-06:00').dt.tz_convert('UTC-05:00')

# 3. Set index
df = df.set_index('Datetime').drop(['Date','Time'], axis=1)

df

,Open,High,Low,Close,Volume
Datetime,,,,,
2007-04-01 18:01:00-05:00,1791.00,1791.00,1790.75,1790.75,11
2007-04-01 18:03:00-05:00,1790.50,1790.50,1789.75,1789.75,3
2007-04-01 18:04:00-05:00,1790.25,1790.25,1790.25,1790.25,6
2007-04-01 18:05:00-05:00,1789.50,1790.25,1789.50,1790.25,4
2007-04-01 18:06:00-05:00,1790.00,1790.50,1790.00,1790.50,5
...,...,...,...,...,...
2025-06-25 00:56:00-05:00,22423.75,22424.50,22423.75,22424.25,20
2025-06-25 00:57:00-05:00,22424.25,22425.25,22423.50,22425.00,25
2025-06-25 00:58:00-05:00,22425.50,22427.00,22425.00,22425.00,27


## Load and prepare the data

### Subtask:
Load the data from "/content/nq-1m.csv", parse the datetime, adjust the timezone, and set the datetime as the index.

**Reasoning**:
Use pandas to read the CSV file, specifying the separator, column names, and header. Combine the 'Date' and 'Time' columns into a single 'Datetime' column, convert it to datetime objects, localize it to 'UTC-06:00', and then convert it to 'UTC-05:00'. Finally, set the 'Datetime' column as the index and drop the original 'Date' and 'Time' columns.

In [13]:
import pandas as pd

csv_path = '/content/nq-1m.csv'
df = pd.read_csv(
    csv_path,
    sep=';',
    names=['Date','Time','Open','High','Low','Close','Volume'],
    header=0
)

df['Datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'], format='%d/%m/%Y %H:%M:%S')
df['Datetime'] = df['Datetime'].dt.tz_localize('UTC-06:00').dt.tz_convert('UTC-05:00')
df = df.set_index('Datetime').drop(['Date','Time'], axis=1)

# Sort the DataFrame by Datetime
df_sorted = df.sort_index()

display(df_sorted.head())

,Open,High,Low,Close,Volume
Datetime,,,,,
2007-04-01 18:01:00-05:00,1791.00,1791.00,1790.75,1790.75,11
2007-04-01 18:03:00-05:00,1790.50,1790.50,1789.75,1789.75,3
2007-04-01 18:04:00-05:00,1790.25,1790.25,1790.25,1790.25,6
2007-04-01 18:05:00-05:00,1789.50,1790.25,1789.50,1790.25,4
2007-04-01 18:06:00-05:00,1790.00,1790.50,1790.00,1790.50,5


## Resample data

### Subtask:
Resample the sorted data to hourly frequency to get hourly open, high, low, and close prices.

**Reasoning**:
Use the `resample()` method with 'H' frequency to aggregate the data hourly. Apply the `first()` aggregation for 'Open', `max()` for 'High', `min()` for 'Low', and `last()` for 'Close' to get the respective hourly values.

In [15]:
hourly_data = df_sorted.resample('h').agg({
    'Open': 'first',
    'High': 'max',
    'Low': 'min',
    'Close': 'last',
    'Volume': 'sum' # Include volume just in case, though not directly used in sweep logic
}).dropna() # Drop any hours with no data

display(hourly_data.head())

,Open,High,Low,Close,Volume
Datetime,,,,,
2007-04-01 18:00:00-05:00,1791.00,1792.00,1788.75,1790.50,116
2007-04-01 19:00:00-05:00,1790.00,1791.75,1789.25,1791.75,115
2007-04-01 20:00:00-05:00,1791.25,1794.00,1791.00,1793.75,305
2007-04-01 21:00:00-05:00,1793.50,1793.75,1792.25,1792.75,57
2007-04-01 22:00:00-05:00,1793.00,1793.25,1792.25,1793.25,39


## Calculate previous hour's range

### Subtask:
Calculate the previous hour's high and low and add these as new columns to the `hourly_data` DataFrame.

**Reasoning**:
Use the `shift()` method to get the previous hour's 'High' and 'Low' values and store them in new columns named 'Prev_High' and 'Prev_Low' in the `hourly_data` DataFrame.

In [16]:
hourly_data['Prev_High'] = hourly_data['High'].shift(1)
hourly_data['Prev_Low'] = hourly_data['Low'].shift(1)
hourly_data = hourly_data.dropna()
display(hourly_data.head())

,Open,High,Low,Close,Volume,Prev_High,Prev_Low
Datetime,,,,,,,
2007-04-01 19:00:00-05:00,1790.00,1791.75,1789.25,1791.75,115,1792.00,1788.75
2007-04-01 20:00:00-05:00,1791.25,1794.00,1791.00,1793.75,305,1791.75,1789.25
2007-04-01 21:00:00-05:00,1793.50,1793.75,1792.25,1792.75,57,1794.00,1791.00
2007-04-01 22:00:00-05:00,1793.00,1793.25,1792.25,1793.25,39,1793.75,1792.25
2007-04-01 23:00:00-05:00,1793.25,1793.50,1793.00,1793.00,59,1793.25,1792.25


## Analyze each hour and record instances

### Subtask:
Iterate through each hour, analyze price movements for sweeps and retracements using minute-level data, and record the results for each instance.

**Reasoning**:
Iterate through each hour in the `hourly_data` DataFrame. For each hour, filter the corresponding minute-level data from `df_sorted`. Check if the current hour's open is within the previous hour's high and low range. If it is, determine if a high or low sweep occurred within the hour by checking the max and min of the minute data. If a sweep occurred, check if the price retraced back to the current hour's open using the minute-level high and low after the sweep time. Record the date, hour, sweep direction (if any), and retracement result (True/False) for each instance where the open was within the previous hour's range.

In [17]:
from collections import defaultdict
import pandas as pd

# List to store detailed results for each instance
instance_results = []

# Dictionary to aggregate results for probability calculation
hourly_analysis = defaultdict(lambda: {'sample_size': 0, 'high_sweep_return': 0, 'low_sweep_return': 0})

# Iterate through each hour in the hourly_data DataFrame
for index, row in hourly_data.iterrows():
    hour = index.hour
    date = index.date() # Get the date of the hour

    prev_high = row['Prev_High']
    prev_low = row['Prev_Low']
    current_open = row['Open']
    current_high = row['High']
    current_low = row['Low']

    # Check if current hour's open is within the previous hour's range
    if prev_low <= current_open <= prev_high:
        hourly_analysis[hour]['sample_size'] += 1 # Increment sample size only when open is within range

        # Get the minute-level data for the current hour
        next_hour_start = index + pd.Timedelta(hours=1)
        # Ensure we only get data within the current hour
        minute_data_this_hour = df_sorted.loc[index : next_hour_start - pd.Timedelta(seconds=1)]

        sweep_direction = None
        retracement_to_open = False

        # Check for high sweep
        if current_high >= prev_high:
            sweep_direction = 'High'
            # Find the time of the high sweep (first time high is >= prev_high)
            high_sweep_time = minute_data_this_hour[minute_data_this_hour['High'] >= prev_high].index.min()
            # Check for retracement to open after high sweep using minute lows
            if not minute_data_this_hour.loc[high_sweep_time:]['Low'].empty and minute_data_this_hour.loc[high_sweep_time:]['Low'].min() <= current_open:
                 retracement_to_open = True
                 hourly_analysis[hour]['high_sweep_return'] += 1


        # Check for low sweep (only if high sweep didn't occur first in this simplified logic)
        # If a high sweep happened, we assume the high sweep scenario takes precedence for this analysis instance
        if sweep_direction is None and current_low <= prev_low:
             sweep_direction = 'Low'
             # Find the time of the low sweep (first time low is <= prev_low)
             low_sweep_time = minute_data_this_hour[minute_data_this_hour['Low'] <= prev_low].index.min()
             # Check for retracement to open after low sweep using minute highs
             if not minute_data_this_hour.loc[low_sweep_time:]['High'].empty and minute_data_this_hour.loc[low_sweep_time:]['High'].max() >= current_open:
                 retracement_to_open = True
                 hourly_analysis[hour]['low_sweep_return'] += 1


        # Record the results for this instance if open was within the previous hour's range
        instance_results.append({
            'Date': date,
            'Hour': hour,
            'Sweep Direction': sweep_direction,
            'Retracement to Open': retracement_to_open
        })

# Display the detailed instance results
instance_results_df = pd.DataFrame(instance_results)
print("Detailed Instance Results:")
display(instance_results_df.head()) # Displaying head to avoid excessive output
print("\n...")
display(instance_results_df.tail()) # Displaying tail

Detailed Instance Results:


,Date,Hour,Sweep Direction,Retracement to Open
0,2007-04-01,19,None,False
1,2007-04-01,20,High,True
2,2007-04-01,21,None,False
3,2007-04-01,22,Low,True
4,2007-04-01,23,High,True



...


,Date,Hour,Sweep Direction,Retracement to Open
107146,2025-06-24,21,Low,False
107147,2025-06-24,22,None,False
107148,2025-06-24,23,High,True
107149,2025-06-25,0,High,False
107150,2025-06-25,1,None,False


## Calculate and display probabilities

### Subtask:
Calculate the probabilities of sweeping and returning to open for both high and low sweeps for each hour and display the results in two tables.

**Reasoning**:
Iterate through the aggregated results for each hour. Calculate the probability of a high sweep with return to open by dividing the count of `high_sweep_return` by the `sample_size`. Similarly, calculate the probability of a low sweep with return to open by dividing the count of `low_sweep_return` by the `sample_size`. Create two pandas DataFrames to store the results for high sweeps and low sweeps, and display them.

In [18]:
import pandas as pd

high_sweep_prob_data = []
low_sweep_prob_data = []

for hour, data in hourly_analysis.items():
    sample_size = data['sample_size']
    high_sweep_return = data['high_sweep_return']
    low_sweep_return = data['low_sweep_return']

    high_sweep_prob = (high_sweep_return / sample_size) if sample_size > 0 else 0
    low_sweep_prob = (low_sweep_return / sample_size) if sample_size > 0 else 0

    high_sweep_prob_data.append({'Hour': hour, 'Sample Size': sample_size, 'Probability of High Sweep and Return to Open': high_sweep_prob})
    low_sweep_prob_data.append({'Hour': hour, 'Sample Size': sample_size, 'Probability of Low Sweep and Return to Open': low_sweep_prob})

high_sweep_prob_df = pd.DataFrame(high_sweep_prob_data).sort_values(by='Hour')
low_sweep_prob_df = pd.DataFrame(low_sweep_prob_data).sort_values(by='Hour')

print("Probability of taking previous hourly high and returning back to hourly open:")
display(high_sweep_prob_df)

print("\nProbability of taking previous hourly low and returning back to hourly open:")
display(low_sweep_prob_df)

Probability of taking previous hourly high and returning back to hourly open:


,Hour,Sample Size,Probability of High Sweep and Return to Open
5,0,4585,0.336096
6,1,4552,0.376757
7,2,4577,0.429757
8,3,4657,0.447069
9,4,4663,0.269354
10,5,4668,0.286204
11,6,4663,0.313532
12,7,4681,0.341807
13,8,4683,0.402093
14,9,4694,0.531104



Probability of taking previous hourly low and returning back to hourly open:


,Hour,Sample Size,Probability of Low Sweep and Return to Open
5,0,4585,0.211778
6,1,4552,0.209798
7,2,4577,0.216954
8,3,4657,0.225467
9,4,4663,0.206519
10,5,4668,0.214225
11,6,4663,0.222175
12,7,4681,0.205298
13,8,4683,0.218236
14,9,4694,0.188752


# Task
Analyze the provided trading data to determine the probability of the price returning to the hourly open after sweeping the previous hour's high or low, broken down by the minute within the hour when the sweep occurs. Provide separate probability tables for high sweeps and low sweeps, showing the hour and the probability for sweep times in the 00-19, 20-39, and 40-59 minute intervals. Also, print all individual instances with date, hour, sweep direction (if occurred), and the result of retracement back to open (True/False). Use the data from "EURUSD_M1_2020.csv".

## Analyze each hour and record instances with sweep time

### Subtask:
Modify the current analysis loop to record the minute within the hour when a high or low sweep occurs, in addition to the existing information.


**Reasoning**:
Modify the analysis loop to include the sweep minute for both high and low sweeps and append it to the instance results.



In [19]:
from collections import defaultdict
import pandas as pd

# List to store detailed results for each instance
instance_results = []

# Dictionary to aggregate results for probability calculation
hourly_analysis = defaultdict(lambda: {'sample_size': 0, 'high_sweep_return': 0, 'low_sweep_return': 0})

# Iterate through each hour in the hourly_data DataFrame
for index, row in hourly_data.iterrows():
    hour = index.hour
    date = index.date() # Get the date of the hour

    prev_high = row['Prev_High']
    prev_low = row['Prev_Low']
    current_open = row['Open']
    current_high = row['High']
    current_low = row['Low']

    sweep_direction = None
    retracement_to_open = False
    sweep_minute = None # Initialize sweep_minute

    # Check if current hour's open is within the previous hour's range
    if prev_low <= current_open <= prev_high:
        hourly_analysis[hour]['sample_size'] += 1 # Increment sample size only when open is within range

        # Get the minute-level data for the current hour
        next_hour_start = index + pd.Timedelta(hours=1)
        # Ensure we only get data within the current hour
        minute_data_this_hour = df_sorted.loc[index : next_hour_start - pd.Timedelta(seconds=1)]

        # Check for high sweep
        if current_high >= prev_high:
            sweep_direction = 'High'
            # Find the time of the high sweep (first time high is >= prev_high)
            high_sweep_time = minute_data_this_hour[minute_data_this_hour['High'] >= prev_high].index.min()
            sweep_minute = high_sweep_time.minute # Store the minute of the high sweep
            # Check for retracement to open after high sweep using minute lows
            if not minute_data_this_hour.loc[high_sweep_time:]['Low'].empty and minute_data_this_hour.loc[high_sweep_time:]['Low'].min() <= current_open:
                 retracement_to_open = True
                 hourly_analysis[hour]['high_sweep_return'] += 1


        # Check for low sweep (only if high sweep didn't occur first in this simplified logic)
        # If a high sweep happened, we assume the high sweep scenario takes precedence for this analysis instance
        if sweep_direction is None and current_low <= prev_low:
             sweep_direction = 'Low'
             # Find the time of the low sweep (first time low is <= prev_low)
             low_sweep_time = minute_data_this_hour[minute_data_this_hour['Low'] <= prev_low].index.min()
             sweep_minute = low_sweep_time.minute # Store the minute of the low sweep
             # Check for retracement to open after low sweep using minute highs
             if not minute_data_this_hour.loc[low_sweep_time:]['High'].empty and minute_data_this_hour.loc[low_sweep_time:]['High'].max() >= current_open:
                 retracement_to_open = True
                 hourly_analysis[hour]['low_sweep_return'] += 1


        # Record the results for this instance if open was within the previous hour's range
        instance_results.append({
            'Date': date,
            'Hour': hour,
            'Sweep Direction': sweep_direction,
            'Retracement to Open': retracement_to_open,
            'Sweep Minute': sweep_minute # Add sweep minute to results
        })

# Display the detailed instance results
instance_results_df = pd.DataFrame(instance_results)
print("Detailed Instance Results:")
display(instance_results_df.head()) # Displaying head to avoid excessive output
print("\n...")
display(instance_results_df.tail()) # Displaying tail

Detailed Instance Results:


,Date,Hour,Sweep Direction,Retracement to Open,Sweep Minute
0,2007-04-01,19,None,False,NaN
1,2007-04-01,20,High,True,1.0
2,2007-04-01,21,None,False,NaN
3,2007-04-01,22,Low,True,22.0
4,2007-04-01,23,High,True,21.0



...


,Date,Hour,Sweep Direction,Retracement to Open,Sweep Minute
107146,2025-06-24,21,Low,False,7.0
107147,2025-06-24,22,None,False,NaN
107148,2025-06-24,23,High,True,0.0
107149,2025-06-25,0,High,False,51.0
107150,2025-06-25,1,None,False,NaN


## Categorize sweep times

### Subtask:
Define the time intervals within the hour (00-19, 20-39, 40-59 minutes).


**Reasoning**:
Define a function to categorize sweep minutes and apply it to the DataFrame.



In [20]:
def categorize_sweep_minute(minute):
    if pd.isna(minute):
        return None
    elif 0 <= minute <= 19:
        return '00-19'
    elif 20 <= minute <= 39:
        return '20-39'
    elif 40 <= minute <= 59:
        return '40-59'
    else:
        return None

instance_results_df['Sweep Time Category'] = instance_results_df['Sweep Minute'].apply(categorize_sweep_minute)

display(instance_results_df.head())

,Date,Hour,Sweep Direction,Retracement to Open,Sweep Minute,Sweep Time Category
0,2007-04-01,19,None,False,NaN,None
1,2007-04-01,20,High,True,1.0,00-19
2,2007-04-01,21,None,False,NaN,None
3,2007-04-01,22,Low,True,22.0,20-39
4,2007-04-01,23,High,True,21.0,20-39


## Aggregate results by hour and sweep time category

### Subtask:
Group the instance results by both the hour of the day and the categorized sweep time. Count the sample size and the number of retracements to open for each group.


**Reasoning**:
Group the instance results by hour, sweep time category, and sweep direction, calculate the sample size and retracement count for each group, reset the index, rename the columns, and filter for rows where a sweep occurred.



In [21]:
# Group by Hour, Sweep Time Category, and Sweep Direction
grouped_results = instance_results_df.groupby(['Hour', 'Sweep Time Category', 'Sweep Direction']).agg(
    sample_size=('Retracement to Open', 'count'),
    retracement_count=('Retracement to Open', 'sum')
).reset_index()

# Filter for rows where a sweep occurred
grouped_sweeps = grouped_results[grouped_results['Sweep Direction'].isin(['High', 'Low'])]

display(grouped_sweeps.head())

,Hour,Sweep Time Category,Sweep Direction,sample_size,retracement_count
0,0,00-19,High,1529,1245
1,0,00-19,Low,1029,793
2,0,20-39,High,500,235
3,0,20-39,Low,352,145
4,0,40-59,High,297,61


**Reasoning**:
Calculate the probabilities of retracement for high and low sweeps within each hour and sweep time category and display the results in two tables.



In [22]:
# Calculate probability
grouped_sweeps['Probability'] = grouped_sweeps['retracement_count'] / grouped_sweeps['sample_size']

# Separate into high and low sweeps
high_sweep_time_prob = grouped_sweeps[grouped_sweeps['Sweep Direction'] == 'High'].pivot_table(
    index='Hour', columns='Sweep Time Category', values='Probability'
)

low_sweep_time_prob = grouped_sweeps[grouped_sweeps['Sweep Direction'] == 'Low'].pivot_table(
    index='Hour', columns='Sweep Time Category', values='Probability'
)

print("Probability of High Sweep and Return to Open by Sweep Time Category:")
display(high_sweep_time_prob)

print("\nProbability of Low Sweep and Return to Open by Sweep Time Category:")
display(low_sweep_time_prob)

Probability of High Sweep and Return to Open by Sweep Time Category:


Sweep Time Category,00-19,20-39,40-59
Hour,,,
0,0.814258,0.470000,0.205387
1,0.785674,0.472656,0.241379
2,0.802705,0.440964,0.230769
3,0.801678,0.484581,0.212670
4,0.730642,0.322314,0.144033
5,0.763212,0.387025,0.180212
6,0.775470,0.428870,0.187879
7,0.748906,0.374207,0.189474
8,0.829003,0.566327,0.248148



Probability of Low Sweep and Return to Open by Sweep Time Category:


Sweep Time Category,00-19,20-39,40-59
Hour,,,
0,0.770651,0.411932,0.208861
1,0.708738,0.450382,0.298246
2,0.741722,0.435233,0.166667
3,0.748103,0.365385,0.140000
4,0.705236,0.324675,0.153846
5,0.754306,0.364198,0.301205
6,0.731544,0.422951,0.210843
7,0.698480,0.386282,0.180000
8,0.760036,0.443243,0.213675


**Reasoning**:
Display all individual instances with date, hour, sweep direction (if occurred), and the result of retracement back to open (True/False).



In [23]:
# Display all individual instances
print("All Individual Instances:")
display(instance_results_df[['Date', 'Hour', 'Sweep Direction', 'Retracement to Open', 'Sweep Time Category']])

All Individual Instances:


,Date,Hour,Sweep Direction,Retracement to Open,Sweep Time Category
0,2007-04-01,19,None,False,None
1,2007-04-01,20,High,True,00-19
2,2007-04-01,21,None,False,None
3,2007-04-01,22,Low,True,20-39
4,2007-04-01,23,High,True,20-39
...,...,...,...,...,...
107146,2025-06-24,21,Low,False,00-19
107147,2025-06-24,22,None,False,None
107148,2025-06-24,23,High,True,00-19
107149,2025-06-25,0,High,False,40-59


## Present the categorized probabilities

### Subtask:
Display the aggregated probabilities of sweeping and returning to open, categorized by hour and sweep time interval, in two separate tables for high and low sweeps.


**Reasoning**:
Display the probability tables for high and low sweeps categorized by sweep time, and display the detailed instance results including the sweep time category.



In [24]:
print("Probability of High Sweep and Return to Open by Sweep Time Category:")
display(high_sweep_time_prob)

print("\nProbability of Low Sweep and Return to Open by Sweep Time Category:")
display(low_sweep_time_prob)

print("\nAll Individual Instances:")
display(instance_results_df[['Date', 'Hour', 'Sweep Direction', 'Retracement to Open', 'Sweep Time Category']])

Probability of High Sweep and Return to Open by Sweep Time Category:


Sweep Time Category,00-19,20-39,40-59
Hour,,,
0,0.814258,0.470000,0.205387
1,0.785674,0.472656,0.241379
2,0.802705,0.440964,0.230769
3,0.801678,0.484581,0.212670
4,0.730642,0.322314,0.144033
5,0.763212,0.387025,0.180212
6,0.775470,0.428870,0.187879
7,0.748906,0.374207,0.189474
8,0.829003,0.566327,0.248148



Probability of Low Sweep and Return to Open by Sweep Time Category:


Sweep Time Category,00-19,20-39,40-59
Hour,,,
0,0.770651,0.411932,0.208861
1,0.708738,0.450382,0.298246
2,0.741722,0.435233,0.166667
3,0.748103,0.365385,0.140000
4,0.705236,0.324675,0.153846
5,0.754306,0.364198,0.301205
6,0.731544,0.422951,0.210843
7,0.698480,0.386282,0.180000
8,0.760036,0.443243,0.213675



All Individual Instances:


,Date,Hour,Sweep Direction,Retracement to Open,Sweep Time Category
0,2007-04-01,19,None,False,None
1,2007-04-01,20,High,True,00-19
2,2007-04-01,21,None,False,None
3,2007-04-01,22,Low,True,20-39
4,2007-04-01,23,High,True,20-39
...,...,...,...,...,...
107146,2025-06-24,21,Low,False,00-19
107147,2025-06-24,22,None,False,None
107148,2025-06-24,23,High,True,00-19
107149,2025-06-25,0,High,False,40-59


## Summary:

### Data Analysis Key Findings

* The analysis successfully calculated the probability of the price returning to the hourly open after sweeping the previous hour's high or low, broken down by the hour of the day and categorized sweep times (00-19, 20-39, and 40-59 minutes within the hour).
* Separate probability tables were generated for high sweeps and low sweeps, showing the calculated probabilities for each hour and sweep time category.
* A detailed list of all individual instances was provided, including the date, hour, sweep direction (if a sweep occurred), whether a retracement to the hourly open happened, and the minute category of the sweep time.

### Insights or Next Steps

* Analyze the resulting probability tables to identify if certain hours of the day or specific time intervals within the hour show a significantly higher or lower probability of retracement to the hourly open after a sweep. This could inform trading strategies based on sweep timing.
* Investigate the instances where no sweep occurred or where a sweep occurred but did not lead to a retracement to the open to understand the characteristics of those scenarios.


In [28]:
low_sweep_time_prob.to_csv('low_sweep_time_prob.csv', index=True)



In [29]:
high_sweep_time_prob.to_csv('high_sweep_time_prob.csv', index=True)


# Task
Analyze the provided financial data to determine the probability of the price returning to the hourly open after sweeping the previous hour's high or low. Categorize these probabilities based on the 5-minute interval within the hour when the sweep occurred. Provide separate tables for high sweeps and low sweeps, showing the hour and the probability of returning to open for each 5-minute sweep time category (00-04, 05-09, ..., 55-59 minutes). Also, print all instances with date, hour, sweep direction (if occurred), and the result of retracement back to open (true/false). Use the data from "all_hour_data.csv".

## Analyze each hour and record instances with sweep time

### Subtask:
Modify the current analysis loop to record the minute within the hour when a high or low sweep occurs, in addition to the existing information.


**Reasoning**:
Modify the existing analysis loop to capture the sweep minute for both high and low sweeps and append it to the instance results.



In [30]:
from collections import defaultdict
import pandas as pd

# List to store detailed results for each instance
instance_results = []

# Dictionary to aggregate results for probability calculation
hourly_analysis = defaultdict(lambda: {'sample_size': 0, 'high_sweep_return': 0, 'low_sweep_return': 0})

# Iterate through each hour in the hourly_data DataFrame
for index, row in hourly_data.iterrows():
    hour = index.hour
    date = index.date() # Get the date of the hour

    prev_high = row['Prev_High']
    prev_low = row['Prev_Low']
    current_open = row['Open']
    current_high = row['High']
    current_low = row['Low']

    sweep_direction = None
    retracement_to_open = False
    sweep_minute = None # Initialize sweep_minute

    # Check if current hour's open is within the previous hour's range
    if prev_low <= current_open <= prev_high:
        hourly_analysis[hour]['sample_size'] += 1 # Increment sample size only when open is within range

        # Get the minute-level data for the current hour
        next_hour_start = index + pd.Timedelta(hours=1)
        # Ensure we only get data within the current hour
        minute_data_this_hour = df_sorted.loc[index : next_hour_start - pd.Timedelta(seconds=1)]

        # Check for high sweep
        if current_high >= prev_high:
            sweep_direction = 'High'
            # Find the time of the high sweep (first time high is >= prev_high)
            high_sweep_time = minute_data_this_hour[minute_data_this_hour['High'] >= prev_high].index.min()
            if high_sweep_time is not pd.NaT:
                sweep_minute = high_sweep_time.minute # Store the minute of the high sweep
                # Check for retracement to open after high sweep using minute lows
                if not minute_data_this_hour.loc[high_sweep_time:]['Low'].empty and minute_data_this_hour.loc[high_sweep_time:]['Low'].min() <= current_open:
                     retracement_to_open = True
                     hourly_analysis[hour]['high_sweep_return'] += 1


        # Check for low sweep (only if high sweep didn't occur first in this simplified logic)
        # If a high sweep happened, we assume the high sweep scenario takes precedence for this analysis instance
        if sweep_direction is None and current_low <= prev_low:
             sweep_direction = 'Low'
             # Find the time of the low sweep (first time low is <= prev_low)
             low_sweep_time = minute_data_this_hour[minute_data_this_hour['Low'] <= prev_low].index.min()
             if low_sweep_time is not pd.NaT:
                sweep_minute = low_sweep_time.minute # Store the minute of the low sweep
                # Check for retracement to open after low sweep using minute highs
                if not minute_data_this_hour.loc[low_sweep_time:]['High'].empty and minute_data_this_hour.loc[low_sweep_time:]['High'].max() >= current_open:
                    retracement_to_open = True
                    hourly_analysis[hour]['low_sweep_return'] += 1


        # Record the results for this instance if open was within the previous hour's range
        instance_results.append({
            'Date': date,
            'Hour': hour,
            'Sweep Direction': sweep_direction,
            'Retracement to Open': retracement_to_open,
            'Sweep Minute': sweep_minute # Add sweep minute to results
        })

# Display the detailed instance results
instance_results_df = pd.DataFrame(instance_results)
print("Detailed Instance Results:")
display(instance_results_df.head()) # Displaying head to avoid excessive output
print("\n...")
display(instance_results_df.tail()) # Displaying tail

Detailed Instance Results:


,Date,Hour,Sweep Direction,Retracement to Open,Sweep Minute
0,2007-04-01,19,None,False,NaN
1,2007-04-01,20,High,True,1.0
2,2007-04-01,21,None,False,NaN
3,2007-04-01,22,Low,True,22.0
4,2007-04-01,23,High,True,21.0



...


,Date,Hour,Sweep Direction,Retracement to Open,Sweep Minute
107146,2025-06-24,21,Low,False,7.0
107147,2025-06-24,22,None,False,NaN
107148,2025-06-24,23,High,True,0.0
107149,2025-06-25,0,High,False,51.0
107150,2025-06-25,1,None,False,NaN


**Reasoning**:
Define a function to categorize sweep minutes into 5-minute intervals and apply it to the DataFrame to create a new column for the sweep time category.



In [31]:
def categorize_sweep_minute_5min(minute):
    if pd.isna(minute):
        return None
    else:
        return f'{int(minute // 5) * 5:02d}-{int(minute // 5) * 5 + 4:02d}'

instance_results_df['Sweep Time Category'] = instance_results_df['Sweep Minute'].apply(categorize_sweep_minute_5min)

display(instance_results_df.head())

,Date,Hour,Sweep Direction,Retracement to Open,Sweep Minute,Sweep Time Category
0,2007-04-01,19,None,False,NaN,None
1,2007-04-01,20,High,True,1.0,00-04
2,2007-04-01,21,None,False,NaN,None
3,2007-04-01,22,Low,True,22.0,20-24
4,2007-04-01,23,High,True,21.0,20-24


**Reasoning**:
Group the instance results by hour, sweep time category, and sweep direction, calculate the sample size and retracement count for each group, reset the index, and filter for rows where a sweep occurred.



In [32]:
# Group by Hour, Sweep Time Category, and Sweep Direction
grouped_results_5min = instance_results_df.groupby(['Hour', 'Sweep Time Category', 'Sweep Direction']).agg(
    sample_size=('Retracement to Open', 'count'),
    retracement_count=('Retracement to Open', 'sum')
).reset_index()

# Filter for rows where a sweep occurred
grouped_sweeps_5min = grouped_results_5min[grouped_results_5min['Sweep Direction'].isin(['High', 'Low'])]

display(grouped_sweeps_5min.head())

,Hour,Sweep Time Category,Sweep Direction,sample_size,retracement_count
0,0,00-04,High,790,736
1,0,00-04,Low,531,478
2,0,05-09,High,318,245
3,0,05-09,Low,230,172
4,0,10-14,High,222,139


**Reasoning**:
Calculate the probabilities of retracement for high and low sweeps within each hour and 5-minute sweep time category and display the results in two tables.



In [33]:
# Calculate probability
grouped_sweeps_5min['Probability'] = grouped_sweeps_5min['retracement_count'] / grouped_sweeps_5min['sample_size']

# Separate into high and low sweeps and pivot for table format
high_sweep_time_prob_5min = grouped_sweeps_5min[grouped_sweeps_5min['Sweep Direction'] == 'High'].pivot_table(
    index='Hour', columns='Sweep Time Category', values='Probability'
)

low_sweep_time_prob_5min = grouped_sweeps_5min[grouped_sweeps_5min['Sweep Direction'] == 'Low'].pivot_table(
    index='Hour', columns='Sweep Time Category', values='Probability'
)

print("Probability of High Sweep and Return to Open by 5-Minute Sweep Time Category:")
display(high_sweep_time_prob_5min)

print("\nProbability of Low Sweep and Return to Open by 5-Minute Sweep Time Category:")
display(low_sweep_time_prob_5min)

Probability of High Sweep and Return to Open by 5-Minute Sweep Time Category:


Sweep Time Category,00-04,05-09,10-14,15-19,20-24,25-29,30-34,35-39,40-44,45-49,50-54,55-59
Hour,,,,,,,,,,,,
0,0.931646,0.770440,0.626126,0.628141,0.522388,0.480916,0.516779,0.290698,0.273810,0.207317,0.156250,0.164179
1,0.911355,0.639394,0.626667,0.575000,0.510067,0.492647,0.456140,0.415929,0.320388,0.306122,0.200000,0.125000
2,0.903159,0.642202,0.579439,0.489796,0.514085,0.453488,0.383178,0.375000,0.320988,0.294118,0.250000,0.055556
3,0.905592,0.695946,0.632231,0.527363,0.578571,0.504202,0.368421,0.456790,0.298507,0.261538,0.069767,0.152174
4,0.863309,0.610561,0.549020,0.511765,0.417476,0.243590,0.353535,0.240964,0.246154,0.137931,0.114754,0.067797
5,0.909988,0.667845,0.529101,0.488506,0.477941,0.431193,0.292683,0.316456,0.228916,0.250000,0.136986,0.084746
6,0.920188,0.647260,0.575221,0.538012,0.485714,0.450820,0.390909,0.367925,0.278351,0.197674,0.128205,0.115942
7,0.894581,0.647696,0.506122,0.492462,0.417910,0.381356,0.381356,0.300971,0.260274,0.246914,0.116667,0.112676
8,0.928810,0.753799,0.681614,0.671569,0.598802,0.630137,0.609890,0.280374,0.333333,0.316456,0.192982,0.042553



Probability of Low Sweep and Return to Open by 5-Minute Sweep Time Category:


Sweep Time Category,00-04,05-09,10-14,15-19,20-24,25-29,30-34,35-39,40-44,45-49,50-54,55-59
Hour,,,,,,,,,,,,
0,0.900188,0.747826,0.548387,0.513274,0.438776,0.494382,0.361111,0.333333,0.224490,0.322581,0.230769,0.076923
1,0.848185,0.587361,0.555556,0.447368,0.505263,0.392857,0.447761,0.409091,0.425000,0.266667,0.241379,0.133333
2,0.853121,0.530612,0.552632,0.395062,0.476190,0.487805,0.371429,0.333333,0.111111,0.230769,0.227273,0.000000
3,0.866513,0.538462,0.466102,0.535714,0.314815,0.395833,0.400000,0.368421,0.157895,0.066667,0.222222,0.142857
4,0.858006,0.585253,0.468750,0.448276,0.398230,0.397260,0.189873,0.255814,0.203704,0.094340,0.193548,0.136364
5,0.892628,0.646766,0.558282,0.469565,0.440678,0.371795,0.289474,0.288462,0.285714,0.390244,0.355556,0.157895
6,0.877138,0.616667,0.596591,0.413534,0.411765,0.441558,0.450704,0.381818,0.294118,0.204545,0.193548,0.125000
7,0.879085,0.567669,0.514970,0.374101,0.484536,0.352941,0.371795,0.254902,0.209302,0.260870,0.147059,0.037037
8,0.898089,0.623762,0.559701,0.515152,0.489362,0.500000,0.427536,0.321429,0.382979,0.142857,0.100000,0.000000


**Reasoning**:
Display all individual instances with date, hour, sweep direction (if occurred), and the result of retracement back to open (True/False), including the 5-minute sweep time category.



In [34]:
# Display all individual instances
print("All Individual Instances:")
display(instance_results_df[['Date', 'Hour', 'Sweep Direction', 'Retracement to Open', 'Sweep Time Category']])

All Individual Instances:


,Date,Hour,Sweep Direction,Retracement to Open,Sweep Time Category
0,2007-04-01,19,None,False,None
1,2007-04-01,20,High,True,00-04
2,2007-04-01,21,None,False,None
3,2007-04-01,22,Low,True,20-24
4,2007-04-01,23,High,True,20-24
...,...,...,...,...,...
107146,2025-06-24,21,Low,False,05-09
107147,2025-06-24,22,None,False,None
107148,2025-06-24,23,High,True,00-04
107149,2025-06-25,0,High,False,50-54


## Present the categorized probabilities (5-minute brackets)

### Subtask:
Display the aggregated probabilities of sweeping and returning to open, categorized by hour and 5-minute sweep time interval, in two separate tables for high and low sweeps.


**Reasoning**:
Display the probability tables for high and low sweeps categorized by 5-minute sweep time intervals to fulfill the subtask requirement.



In [35]:
print("Probability of High Sweep and Return to Open by 5-Minute Sweep Time Category:")
display(high_sweep_time_prob_5min)

print("\nProbability of Low Sweep and Return to Open by 5-Minute Sweep Time Category:")
display(low_sweep_time_prob_5min)

Probability of High Sweep and Return to Open by 5-Minute Sweep Time Category:


Sweep Time Category,00-04,05-09,10-14,15-19,20-24,25-29,30-34,35-39,40-44,45-49,50-54,55-59
Hour,,,,,,,,,,,,
0,0.931646,0.770440,0.626126,0.628141,0.522388,0.480916,0.516779,0.290698,0.273810,0.207317,0.156250,0.164179
1,0.911355,0.639394,0.626667,0.575000,0.510067,0.492647,0.456140,0.415929,0.320388,0.306122,0.200000,0.125000
2,0.903159,0.642202,0.579439,0.489796,0.514085,0.453488,0.383178,0.375000,0.320988,0.294118,0.250000,0.055556
3,0.905592,0.695946,0.632231,0.527363,0.578571,0.504202,0.368421,0.456790,0.298507,0.261538,0.069767,0.152174
4,0.863309,0.610561,0.549020,0.511765,0.417476,0.243590,0.353535,0.240964,0.246154,0.137931,0.114754,0.067797
5,0.909988,0.667845,0.529101,0.488506,0.477941,0.431193,0.292683,0.316456,0.228916,0.250000,0.136986,0.084746
6,0.920188,0.647260,0.575221,0.538012,0.485714,0.450820,0.390909,0.367925,0.278351,0.197674,0.128205,0.115942
7,0.894581,0.647696,0.506122,0.492462,0.417910,0.381356,0.381356,0.300971,0.260274,0.246914,0.116667,0.112676
8,0.928810,0.753799,0.681614,0.671569,0.598802,0.630137,0.609890,0.280374,0.333333,0.316456,0.192982,0.042553



Probability of Low Sweep and Return to Open by 5-Minute Sweep Time Category:


Sweep Time Category,00-04,05-09,10-14,15-19,20-24,25-29,30-34,35-39,40-44,45-49,50-54,55-59
Hour,,,,,,,,,,,,
0,0.900188,0.747826,0.548387,0.513274,0.438776,0.494382,0.361111,0.333333,0.224490,0.322581,0.230769,0.076923
1,0.848185,0.587361,0.555556,0.447368,0.505263,0.392857,0.447761,0.409091,0.425000,0.266667,0.241379,0.133333
2,0.853121,0.530612,0.552632,0.395062,0.476190,0.487805,0.371429,0.333333,0.111111,0.230769,0.227273,0.000000
3,0.866513,0.538462,0.466102,0.535714,0.314815,0.395833,0.400000,0.368421,0.157895,0.066667,0.222222,0.142857
4,0.858006,0.585253,0.468750,0.448276,0.398230,0.397260,0.189873,0.255814,0.203704,0.094340,0.193548,0.136364
5,0.892628,0.646766,0.558282,0.469565,0.440678,0.371795,0.289474,0.288462,0.285714,0.390244,0.355556,0.157895
6,0.877138,0.616667,0.596591,0.413534,0.411765,0.441558,0.450704,0.381818,0.294118,0.204545,0.193548,0.125000
7,0.879085,0.567669,0.514970,0.374101,0.484536,0.352941,0.371795,0.254902,0.209302,0.260870,0.147059,0.037037
8,0.898089,0.623762,0.559701,0.515152,0.489362,0.500000,0.427536,0.321429,0.382979,0.142857,0.100000,0.000000


## Summary:

### Data Analysis Key Findings

*   The probability of a high sweep returning to the open price varies significantly depending on the hour of the day and the specific 5-minute interval within the hour when the sweep occurs. Some hours and time intervals show probabilities close to 0, while others are closer to 1.
*   Similarly, the probability of a low sweep returning to the open price also varies across different hours and 5-minute sweep time categories.
*   Separate tables were generated and displayed for high sweeps and low sweeps, providing a detailed breakdown of these probabilities across hours and 5-minute intervals (00-04, 05-09, ..., 55-59).
*   All individual instances analyzed, including the date, hour, sweep direction (if a sweep occurred), whether it retraced back to the open, and the 5-minute sweep time category, were printed.

### Insights or Next Steps

*   Analyze specific hours or 5-minute intervals that show consistently high or low probabilities of retracement to identify potential trading strategies or patterns.
*   Investigate if there are specific daily or weekly patterns in the sweep time probabilities across different hours.


In [36]:
high_sweep_time_prob_5min.to_csv('high_sweep_time_prob_5min.csv', index=True)
low_sweep_time_prob_5min.to_csv('low_sweep_time_prob_5min.csv', index=True)
